In [ ]:
import os
import openai
import json
import re
from string import Template
import random

print('Enter OpenAI API key:')
openai.api_key = input()

os.environ['OPENAI_API_KEY']=openai.api_key

In [ ]:
from openai import OpenAI
client = OpenAI()

In [ ]:
#reocurring function used to save GPT responses as JSONs
def save_json(model_output, filename):
  output_text = model_output.strip()

  #getting rid of the ```json
  if output_text.startswith("```"):
    output_text = re.sub(r"^```(?:json)?\s*", "", output_text)
    output_text = re.sub(r"```$", "", output_text.strip())

  try:
    data = json.loads(output_text)
  except json.JSONDecodeError as e:
    print("Error:", e)
    #even if it's not in json format it will save it as a .txt file
    with open(filename, "w", encoding="utf-8") as f:
        f.write(output_text)
  else:
    with open(filename, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4, ensure_ascii=False)
    print(f"Saved valid JSON as {filename}")

In [ ]:
#recurring function used to get all events
def get_events():
  events = []
  with open("table6.json", "r", encoding="utf-8") as f:
    plot_data = json.load(f)

  for node in plot_data.get("nodes", {}).values():
    events.extend(node.get("edgeEvents", []))

  return events

In [ ]:
#for now using the .txt files but will later need to fix this to use the fanfics.csv
fanfic_path = "/content/debug_fic_71802996.txt"
with open(fanfic_path, 'r', encoding='utf-8') as file:
  fanfic = file.read()

In [ ]:
#need to get the plot summaries for the fanfictions
prompt = f"""
Tell me who the main character is on the first line in the form of 'char_name = [character name]' and generate a plot summary of {fanfic} in the form of 'summary = [summary]'
  """
from openai import OpenAI
client = OpenAI()

response = client.responses.create(
    model="gpt-4.1-nano",
    input= prompt
  )
print(response.output_text)
char_and_sum = "character_and_summary.txt"
with open(char_and_sum, "w", encoding="utf-8") as file:
  file.write(response.output_text)

In [ ]:
#this is the JSON schema for generating the tree(plot-to-tree)
JSON_SCHEMA = {
    "nodes": {
        "node_1": {
            "state": "<initial state of {char_name}>",  # The initial state of the main character (no major plot points).
            "goal": "<goal of {char_name} given the current state>",  # Starts with 'To ...'
            "decision": "<key decision taken by {char_name} that propels the story forward>",  # Starts with '{char_name} decides to ...'
            "edgeEvents": [
                "<repeat key decision taken by {char_name} that propels the story forward, starting with '{char_name} decides to ...'>",
                "<event resulting from the key decision and leading to next state>",
                "<next state of {char_name} resulting from the previous events>"
            ],
            "alternate_decision": "<an alternate decision {char_name} could have made given the same state and goal>"
        },
        "node_2": {
            "state": "<state of {char_name} resulting from the previous node's edgeEvents>",  # The current state of the main character.
            "goal": "<goal of {char_name} given the current state>",  # Starts with 'To ...'
            "decision": "<key decision taken by {char_name} that propels the story forward>",  # Starts with '{char_name} decides to ...'
            "edgeEvents": [
                "<repeat key decision taken by {char_name} that propels the story forward, starting with '{char_name} decides to ...'>",
                "<event resulting from the key decision and leading to next state>",
                "<next state of {char_name} resulting from the previous events>"
            ],
            "alternate_decision": "<an alternate decision {char_name} could have made given the same state and goal>"
        },
        "node_3": {
            "state": "<state of {char_name} resulting from the previous node's edgeEvents>",  # The current state of the main character.
            "goal": "<goal of {char_name} given the current state>",  # Starts with 'To ...'
            "decision": "<key decision taken by {char_name} that propels the story forward>",  # Starts with '{char_name} decides to ...'
            "edgeEvents": [
                "<repeat key decision taken by {char_name} that propels the story forward, starting with '{char_name} decides to ...'>",
                "<event resulting from the key decision and leading to next state>",
                "<next state of {char_name} resulting from the previous events>"
            ],
            "alternate_decision": "<an alternate decision {char_name} could have made given the same state and goal>"
        },
        "node_4": {
            "state": "<state of {char_name} resulting from the previous node's edgeEvents>",  # The current state of the main character.
            "goal": "<goal of {char_name} given the current state>",  # Starts with 'To ...'
            "decision": "<key decision taken by {char_name} that propels the story forward>",  # Starts with '{char_name} decides to ...'
            "edgeEvents": [
                "<repeat key decision taken by {char_name} that propels the story forward, starting with '{char_name} decides to ...'>",
                "<event resulting from the key decision and leading to next state>",
                "<next state of {char_name} resulting from the previous events>"
            ],
            "alternate_decision": "<an alternate decision {char_name} could have made given the same state and goal>"
        },
        "node_5": {
            "state": "<state of {char_name} resulting from the previous node's edgeEvents>",  # The final state setup.
            "goal": "<final character goal given the current state>",  # Starts with 'To ...'
            "decision": "<key decision taken by {char_name} that propels the story forward>",  # Starts with '{char_name} decides to ...'
            "edgeEvents": [
                "<repeat key decision taken by {char_name} leading to end of story, starting with '{char_name} decides to ...'>",
                "<event resulting from the key decision and leading to end of story>",
                "<final state of {char_name} resulting from the previous events>"
            ],
            "alternate_decision": "<an alternate decision {char_name} could have made given the same state and goal>"
        },
    }
}

In [ ]:
file_path = '/content/character_and_summary.txt'
#need to get the plot summary and character name generated from .txt
variables = {}
with open(file_path, "r") as f:
       for line in f:
           parts = line.strip().split("=", 1)
           if len(parts) == 2:
               key = parts[0].strip()
               value = parts[1].strip()
               variables[key] = value


print(f"char_name: {variables['char_name']}")
print(f"summary: {variables['summary']}")
num_nodes = 5

#prompting for generating the tree (plot-to-tree)
messages2 = [
    {"role": "system", "content": "You are a helpful fiction writer assistant."},
    {"role": "user", "content": f"""{variables['summary']}\n Summarize the plot above into a plot tree of
    {'at most 6' if num_nodes == '' else num_nodes}
nodes with each node containing the state and goal of {variables['char_name']}, and the key
decision that propels the story forward. Each edge should contain a list of
events that lead {variables['char_name']} to the state of next node. Also, Given the same
state and goal of {variables['char_name']}, imagine an alternate decision that would have led
{variables['char_name']} to a different storyline. Output in JSON format with schema:
{JSON_SCHEMA}. Make sure that all important plot points are included in
'edgeEvents' but not in 'state'"""
}
]

response = client.chat.completions.create(
        model="gpt-4o",
        messages= messages2
    )
print(response.choices[0].message.content)
save_json(response.choices[0].message.content, "table6.json")

Now need to get events from the nodes to do the next prompt

In [ ]:
#JSON schema for extracting key events
JSON_SCHEMA = {
    "inciting_incident": {
        "eventId": "<the event number>",
        "event": "<the event corresponding to the inciting incident>"
    },
    "crisis": {
        "eventId": "<the event number>",
        "event": "<the event corresponding to the crisis>"
    },
    "climax": {
        "eventId": "<the event number>",
        "event": "<the event corresponding to the climax>"
    }
}

In [ ]:
#loading in the tree json and looping through the edgeEvents to find the key events
events = get_events()
events_text = "\n".join(events)
print("events_text:", events_text)

messages2 = [
    {"role": "system","content": f"""Here are some definitions in the context of three-act story
structure: The inciting incident is an event that pulls the protagonist
out of their normal world and into the main action of the story. It is
the turning point between Act One and Act Two. The crisis is the moment
when the protagonist faces their greatest challenge or obstacle, leading
directly to the climax of the story. It is the turning point between Act Two
and Act Three. The climax is the climactic confrontation in which the hero
faces a point of no return: they must either prevail or perish. It occurs in
Act Three and should have the peak tension of the story. You will be given a
list of events from a movie plot. Your task is to identify the inciting
incident, crisis, and climax. Output in JSON format with schema:
{JSON_SCHEMA}."""
},
{
"role": "user", "content": f"{events_text}"
}
]

response = client.chat.completions.create(
        model="gpt-4o",
        messages= messages2
    )
print(response.choices[0].message.content)
save_json(response.choices[0].message.content, "table7.json")

Therefore, we prompt (Table 7) GPT-4 to identify three key events corresponding to the three
major plot points ( 2 in Figure 3) from all of the
edge events in a storyline and save them for later
use in the prompt generation.
Generating a prompt for each node

In [ ]:
#JSON schema for the meta prompting
JSON_SCHEMA = {
    "branching_event_number": "<branching_event>",
    "original_decision": "<storyline[f'node_{branching_node}']['decision']>",
    "alternate_decision": "<storyline[f'node_{branching_node}']['alternate_decision']>",
    "events_to_generate": "<events_to_generate[i]>",
    "major_plot_points": "<mpp>",
    "prompt": (
        "TODO: <a prompt for ChatGPT for every branching point above with the following requirements:\n"
        "1. Ask to use the original storyline as a reference to write an alternate "
        "storyline that branches out at event {branching_event} if {char_name} "
        "{storyline[f'node_{branching_node}']['alternate_decision']} instead of "
        "{storyline[f'node_{branching_node}']['decision']}.\n"
        "2. Provide 5 thought-provoking, concrete guiding questions as potential directions "
        "to explore that expand the following:\n"
        "   a. How would the alternate decision change or replace {mpp}?\n"
        "   b. How would {char_name} make key decisions that overcome new challenges and propel the story forward?\n"
        "3. Describe what an ideal alternate storyline should look like.\n"
        "4. Ask to output the alternate storyline as a list of "
        "{(len(storyline) - branching_node + 1) * 3} events that has "
        "{storyline[f'node_{branching_node}']['alternate_decision']} as the first event.>"
    )
}

In [ ]:
#for this prompting need to define branching_event, branching_node,
#mpp (is it all or just 1), storyline, branching_node value?
"""
With meta-prompts generated for each node of the
original storyline, the system recursively branches
out at each node and writes the alternate storylines
accordingly:
"""
events_to_generate = [15, 12, 9, 6, 3]
branching = [2, 5, 8, 11, 14]
with open("table6.json", "r", encoding="utf-8") as f:
    storyline = json.load(f)

with open("table7.json", "r", encoding="utf-8") as f:
    key_events = json.load(f)

events = get_events()
events_text = "\n".join(events)
print("events_text:", events_text)

inciting_incident_num = int(key_events["inciting_incident"]["eventId"])
crisis_num = int(key_events["crisis"]["eventId"])
climax_num = int(key_events["climax"]["eventId"])

for i in range(num_nodes):
  print(f"should be {(5 - i) *3} new events for node {i+1}")
  #i is the branching_node
  #there are 15 events because there are 5 nodes
  #confused if branching_event is correct
  #random_integer = random.randint(0, 2)
  #branching_event = events[i+4]
  branching_event = events[branching[i]]

  decision = storyline["nodes"][f"node_{i+1}"]["decision"]
  alternate_decision = storyline["nodes"][f"node_{i+1}"]["alternate_decision"]

  if i < inciting_incident_num:
    mpp = key_events["inciting_incident"]["event"]
  elif i < crisis_num:
    mpp = key_events["crisis"]["event"]
  elif i < climax_num:
    mpp = key_events["climax"]["event"]
  else:
    print("print")
  messages2 = [
        {"role": "system", "content": "You are an expert in prompting ChatGPT."},
        {
            "role": "user",
            "content": f"""Original storyline:
{events_text}

Write a prompt for ChatGPT with the following requirements:
1. Ask to use the original storyline as a reference to write an alternate storyline that branches out at event {branching[i]}: '{branching_event}' if {variables['char_name']} {alternate_decision} instead of {decision}.
2. Provide 5 thought-provoking concrete guiding questions as potential directions to explore that expand the following:
   a. How would the alternate decision change or replace {mpp}?
   b. How would {variables['char_name']} make key decisions that overcome new challenges and propel the story forward?
3. Describe what an ideal alternate storyline should look like.
4. Ask to output the alternate storyline as a list of {events_to_generate[i]} events that has {alternate_decision} as the first event.

Output the prompt with the following JSON schema: {JSON_SCHEMA}
"""
        }
    ]

  response = client.chat.completions.create(
        model="gpt-4o",
        messages=messages2
    )

  print(response.choices[0].message.content)
  save_json(response.choices[0].message.content, f"table8_node{i}.json")

  #need to fix the new_story_length and branching_event number
  #hard coding the new_story_length to be 3 events for now because confused on the logic
  #need to review that jsons are outputted properly

In [ ]:
directory = "."
pattern = re.compile(r"(list of\s+)(\d+)(\s+events)", re.IGNORECASE)
expected_counts = {
    1: 15,
    2: 12,
    3: 9,
    4: 6,
    5: 3
}

for filename in os.listdir(directory):
    if filename.startswith("table8_node") and filename.endswith(".json"):
        filepath = os.path.join(directory, filename)
        try:
            with open(filepath, "r", encoding="utf-8") as f:
                data = json.load(f)
        except json.JSONDecodeError as e:
            #some of the meta-prompt outputs are not in JSON
            #so manually going through and fixing them
            with open(filepath, "r", encoding="utf-8") as f:
                raw_text = f.read()
            repaired_text = re.sub(r"(?<!\\)'", '"', raw_text)
            try:
                data = json.loads(repaired_text)
                print(f"Fixed JSON formatting in {filename}")
            except json.JSONDecodeError as e2:
                print(f"Still invalid after fix in {filename}: {e2}")
                continue

        #checking if the events_to_generate is correct because
        #most of the time it is but sometimes for some reason it
        #doubles or does not save a numerical input. It updates
        #both the prompt AND the actual events_to_generate field
        node_match = re.search(r"table8_node(\d+)", filename)

        node_num = int(node_match.group(1)) + 1
        expected_value = str(expected_counts.get(node_num, ""))
        prompt = data.get("prompt", "")

        match = pattern.search(prompt)
        current_value = match.group(2)

        if current_value != expected_value:
            new_prompt = pattern.sub(rf"\g<1>{expected_value}\g<3>", prompt)
            data["prompt"] = new_prompt
            data["events_to_generate"] = expected_value
            print(f"Corrected event count in {filename}: {current_value} → {expected_value}")
        else:
            print(f"Event count already correct ({expected_value})")

        #resaving as JSON
        with open(filepath, "w", encoding="utf-8") as f:
            json.dump(data, f, indent=4, ensure_ascii=False)
            f.write("\n")
            print(f"Saved valid JSON: {filename}")

In [ ]:
#JSON for writing the new storyline
JSON_SCHEMA0 = {
"events": {
"event 1": "<an event in the new storyline>",
"event 2": "<an event in the new storyline>",
"event 3": "<an event in the new storyline>",
"event 4": "<an event in the new storyline>",
"event 5": "<an event in the new storyline>",
"event 6": "<an event in the new storyline>",
"event 7": "<an event in the new storyline>",
"event 8": "<an event in the new storyline>",
"event 9": "<an event in the new storyline>",
"event 10": "<an event in the new storyline>",
"event 11": "<an event in the new storyline>",
"event 12": "<an event in the new storyline>",
"event 13": "<an event in the new storyline>",
"event 14": "<an event in the new storyline>",
"event 15": "<an event in the new storyline>"
}
}

#JSON for writing the new storyline
JSON_SCHEMA1 = {
"events": {
"event 1": "<an event in the new storyline>",
"event 2": "<an event in the new storyline>",
"event 3": "<an event in the new storyline>",
"event 4": "<an event in the new storyline>",
"event 5": "<an event in the new storyline>",
"event 6": "<an event in the new storyline>",
"event 7": "<an event in the new storyline>",
"event 8": "<an event in the new storyline>",
"event 9": "<an event in the new storyline>",
"event 10": "<an event in the new storyline>",
"event 11": "<an event in the new storyline>",
"event 12": "<an event in the new storyline>"
}
}

#JSON for writing the new storyline
JSON_SCHEMA2 = {
"events": {
"event 1": "<an event in the new storyline>",
"event 2": "<an event in the new storyline>",
"event 3": "<an event in the new storyline>",
"event 4": "<an event in the new storyline>",
"event 5": "<an event in the new storyline>",
"event 6": "<an event in the new storyline>",
"event 7": "<an event in the new storyline>",
"event 8": "<an event in the new storyline>",
"event 9": "<an event in the new storyline>"
}
}

#JSON for writing the new storyline
JSON_SCHEMA3 = {
"events": {
"event 1": "<an event in the new storyline>",
"event 2": "<an event in the new storyline>",
"event 3": "<an event in the new storyline>",
"event 4": "<an event in the new storyline>",
"event 5": "<an event in the new storyline>",
"event 6": "<an event in the new storyline>"
}
}

#JSON for writing the new storyline
JSON_SCHEMA4 = {
"events": {
"event 1": "<an event in the new storyline>",
"event 2": "<an event in the new storyline>",
"event 3": "<an event in the new storyline>"
}
}

In [ ]:
events = get_events()
all_events = "\n".join(events)
print("events_text:", all_events)

for i in range(5):
  with open(f"table8_node{i}.json", "r", encoding="utf-8") as f:
      data = json.load(f)
  prompt = data['prompt']

  if i==0:
    messages2 = [
      {"role": "system", "content": "You are a helpful fiction writer assistant."},
      {"role": "user", "content": f"""Original storyline:\n {all_events}\n\n
      {prompt}\n
      Output in JSON format with schema: {JSON_SCHEMA0}."""
      }
      ]

    response = client.chat.completions.create(
          model="gpt-4o",
          messages=messages2
      )

    print(response.choices[0].message.content)
    save_json(response.choices[0].message.content, f"table9_node{i}.json")

  if i==1:
    messages2 = [
      {"role": "system", "content": "You are a helpful fiction writer assistant."},
      {"role": "user", "content": f"""Original storyline:\n {all_events}\n\n
      {prompt}\n
      Output in JSON format with schema: {JSON_SCHEMA1}."""
      }
      ]

    response = client.chat.completions.create(
          model="gpt-4o",
          messages=messages2
      )

    print(response.choices[0].message.content)
    save_json(response.choices[0].message.content, f"table9_node{i}.json")

  if i==2:
    messages2 = [
      {"role": "system", "content": "You are a helpful fiction writer assistant."},
      {"role": "user", "content": f"""Original storyline:\n {all_events}\n\n
      {prompt}\n
      Output in JSON format with schema: {JSON_SCHEMA2}."""
      }
      ]

    response = client.chat.completions.create(
          model="gpt-4o",
          messages=messages2
      )

    print(response.choices[0].message.content)
    save_json(response.choices[0].message.content, f"table9_node{i}.json")

  if i==3:
    messages2 = [
      {"role": "system", "content": "You are a helpful fiction writer assistant."},
      {"role": "user", "content": f"""Original storyline:\n {all_events}\n\n
      {prompt}\n
      Output in JSON format with schema: {JSON_SCHEMA3}."""
      }
      ]

    response = client.chat.completions.create(
          model="gpt-4o",
          messages=messages2
      )

    print(response.choices[0].message.content)
    save_json(response.choices[0].message.content, f"table9_node{i}.json")

  if i==4:
    messages2 = [
      {"role": "system", "content": "You are a helpful fiction writer assistant."},
      {"role": "user", "content": f"""Original storyline:\n {all_events}\n\n
      {prompt}\n
      Output in JSON format with schema: {JSON_SCHEMA4}."""
      }
      ]

    response = client.chat.completions.create(
          model="gpt-4o",
          messages=messages2
      )

    print(response.choices[0].message.content)
    save_json(response.choices[0].message.content, f"table9_node{i}.json")

In [ ]:
#this extracts the JSON from the output of the metaprompting and saves it
#and basically copying the logic from above to clean the JSON and save it
#(will make that a function later)
directory = "."
expected_counts = {
    1: 15,
    2: 12,
    3: 9,
    4: 6,
    5: 3
}

def extract_json_block(text):
    match = re.search(r'\{[\s\S]*\}', text)
    if match:
        json_str = match.group(0)
        try:
            return json.loads(json_str)
        except json.JSONDecodeError:
            fixed = json_str.replace("'", '"')
            fixed = re.sub(r",\s*}", "}", fixed)
            fixed = re.sub(r",\s*]", "]", fixed)
            try:
                return json.loads(fixed)
            except Exception:
                return None
    return None


for filename in os.listdir(directory):
    if filename.startswith("table9_node") and filename.endswith(".json"):
        filepath = os.path.join(directory, filename)

        with open(filepath, "r", encoding="utf-8") as f:
            raw_text = f.read()

        data = extract_json_block(raw_text)
        if data is None:
            repaired_text = re.sub(r"(?<!\\)'", '"', raw_text)
            try:
                data = json.loads(repaired_text)
                print(f"Fixed JSON formatting in {filename}")
            except json.JSONDecodeError as e2:
                print(f"Still invalid after fix in {filename}: {e2}")
                continue

        with open(filepath, "w", encoding="utf-8") as f:
            json.dump(data, f, indent=4, ensure_ascii=False)
            f.write("\n")
            print(f"Saved valid JSON: {filename}")

In [ ]:
#JSON schema for the actual game output
#starting to wonder if the plot summary step is unhelpful
#and if it would be better to just feed the entire fanfic
#into the plot to tree generation step
JSON_SCHEMA = {
    "paragraphs": "<Narrate the three events as three short paragraphs using second-person perspective, then transit to the state and goal of the player. Use newline characters between paragraphs>",
    "button_text_1": "<short button text for original decision>",
    "button_text_2": "<short button text for alternate decision>"
}

In [ ]:
counter = 0
with open("table6.json", "r", encoding="utf-8") as f:
    plot_data = json.load(f)

for node in plot_data.get("nodes", {}).values():
  #print(f"this is node:{node}")
  print(f"this is for node{counter}")
  messages2 = [
          {
              "role": "system",
              "content": f"""You are writing a Choose Your Own Adventure style
  interactive fiction game in which the player is {variables['char_name']}.
  You will be given a list of events, the resulting state and goal of the
  character, and two decisions.

  Do the following:
  1. Narrate each event in a paragraph. You should never mention {variables['char_name']} but always use the second-person perspective.
  2. Seamlessly transition to the state and goal of the player.
  3. Provide two short button texts reflecting the two decisions.
  Output in JSON with schema: {JSON_SCHEMA}"""
          },
          {
              "role": "user",
              "content": f"""{node}"""
          }
      ]
  response = client.chat.completions.create(
          model="gpt-4o",
          messages=messages2
      )


  print(response.choices[0].message.content)
  save_json(response.choices[0].message.content, f"table10_node{counter}.json")
  counter+= 1